# CoNAL (Common Noise Adaptation Layer) and CrowdLayer

In [1]:
import numpy as np
from pathlib import Path

In [2]:
DIR = Path().cwd()
DIRlabelme = (DIR / ".." / "datasets" / "labelme").resolve()
DIR_module = DIRlabelme / "labelme.py"
DIR_answers = DIRlabelme / "answers.json"
print(DIRlabelme)

/home/tlefort/Documents/peerannot/peerannot/datasets/labelme


# Running CoNAL CLI

In [3]:
! peerannot aggregate-deep -h

Usage: peerannot aggregate-deep [OPTIONS] [DATASET]

  Crowdsourcing strategy using deep learning models

Options:
  -K, --n-classes INTEGER   Number of classes to separate
  -o, --output-name TEXT    Name of the generated results file
  -s, --strategy TEXT       Deep learning strategy
  --model TEXT              Neural network to train on
  --answers PATH            Crowdsourced labels in json file
  --img-size INTEGER        Size of image (square)
  --pretrained              Use torch available weights to initialize the
                            network  [default: False]
  --n-epochs INTEGER        Number of training epochs
  --lr FLOAT                Learning rate
  --momentum FLOAT          Momentum for the optimizer
  --decay FLOAT             Weight decay for the optimizer
  --scheduler               Use a multistep scheduler for the learning rate
                            [default: False]
  -m, --milestones INTEGER  Milestones for the learning rate decay scheduler
  --n-para

The following command is an example on how to train CoNAL on labelme, but it is not advised to fully train the network here (long and costly)

```bash
! peerannot aggregate-deep $DIRlabelme -o labelme_conal --answers $DIR_answers -s conal[scale=1e-5] \
    --model modellabelme --img-size=224 --pretrained --n-classes=8 \
    --n-epochs=1000 --lr=5e-3 --optimizer=adam \
    --batch-size=32 --num-workers=8 --data-augmentation
```

# Loading trained CoNAL

In [ ]:
import torch
from peerannot.models.CoNAL import CoNAL_net
from peerannot.helpers.networks import networks
clf = networks("modellabelme", n_classes=8)

In [ ]:
checkpoint = torch.load(DIRlabelme / "best_models" / "labelme_conal.pth")
conal_our = CoNAL_net(input_dim=int(224 * 3 * 224),
                  n_class=8,
                  n_annotator=59,
                  classifier=clf,
                  annotator_dim=59,
                  embedding_dim=20)
conal_our.classifier.load_state_dict(checkpoint["classifier"])
conal_our.auxiliary_network.load_state_dict(checkpoint["auxiliary"])
conal_our.noise_adaptation_layer.load_state_dict(checkpoint["noise_adaptation"])

# Majority vote and load dataset

In [ ]:
! peerannot aggregate $DIRlabelme -s MV

In [ ]:
from peerannot.runners.train import load_all_data
labels_path = DIRlabelme / "labels" / "labels_labelme_mv.npy"
trainset, valset, testset = load_all_data(
    DIRlabelme, labels_path, path_remove=None, labels=labels_path, img_size=224, data_augmentation=False)

# Visualize confusion matrices with CoNAL

In [ ]:
from torchvision import datasets
dataset = datasets.ImageFolder(DIRlabelme / "train")
classes = dataset.class_to_idx

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
conf_local_our = conal_our.noise_adaptation_layer.local_confusion_matrices.detach(
).numpy()  # Confusion matrices of each annotator
conf_global_our = conal_our.noise_adaptation_layer.global_confusion_matrix.detach(
).numpy()  # Confusion matrix of common noise

In [ ]:
list(classes.keys())

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(conf_global_our, cmap="hot")
axs[0].set_title("Global weights learned")
axs[0].xaxis.set_ticks(range(8))
axs[0].xaxis.set_ticklabels( list(classes.keys()), rotation=45)
axs[0].yaxis.set_ticks(range(8))
axs[0].yaxis.set_ticklabels( list(classes.keys()), rotation=0)

axs[1].imshow(conf_local_our[0], cmap="hot")
axs[1].set_title("Weights worker 0")
axs[1].xaxis.set_ticks(range(8))
axs[1].xaxis.set_ticklabels( list(classes.keys()), rotation=45)
axs[1].yaxis.set_ticks([0])
axs[1].yaxis.set_ticklabels("")
fig.suptitle("Our")
plt.show()

# Crowdlayer

In [ ]:
! peerannot agginfo

In [5]:
! peerannot aggregate-deep {DIRlabelme} -o labelme_crowdlayer --answers {DIR_answers} -s CrowdLayer \
    --model resnet18 --img-size=224 --pretrained --n-classes=8 \
    --n-epochs=100 --lr=0.005 -m 1000 -m 1050 --scheduler \
    --batch-size=64 --optimizer=adam --num-workers=8 --data-augmentation

Running the following configuration:
----------
- Data at /home/tlefort/Documents/peerannot/peerannot/datasets/labelme will be saved with prefix labelme_crowdlayer
- number of classes: 8
- output_name: labelme_crowdlayer
- answers: /home/tlefort/Documents/peerannot/peerannot/datasets/labelme/answers.json
- strategy: CrowdLayer
- model: resnet18
- img_size: 224
- pretrained: True
- n_classes: 8
- n_epochs: 100
- lr: 0.005
- milestones: (1000, 1050)
- scheduler: True
- batch_size: 64
- optimizer: adam
- num_workers: 8
- data_augmentation: True
- dataset: /home/tlefort/Documents/peerannot/peerannot/datasets/labelme
- momentum: 0.9
- decay: 0.0005
- n_params: 3072
- lr_decay: 0.1
- path_remove: None
- metadata_path: None
----------
Loading datasets
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Successfully loaded resnet18 

# On CIFAR-10H

In [4]:
DIR = Path().cwd()
DIRc10H = (DIR / ".." / "datasets" / "cifar10H" ).resolve()
DIR_module = DIRc10H / "cifar10h.py"
DIR_answers = DIRc10H / "answers.json"
print(DIRc10H)

/home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H


In [5]:
!peerannot aggregate-deep "/home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H" \
            -o cifar10h_CoNAL[scale=0]_1 \
            --answers=$DIR_answers --model=resnet18 --n-classes=10 --n-epochs 500 \
            --lr 0.1 --optimizer sgd --batch-size 64 --num-workers 8 --img-size=32 \
            --strategy=CoNAL[scale=0] --scheduler -m 250 -m 400

Running the following configuration:
----------
- Data at /home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H will be saved with prefix cifar10h_CoNAL[scale=0]_1
- number of classes: 10
- output_name: cifar10h_CoNAL[scale=0]_1
- answers: /home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H/answers.json
- model: resnet18
- n_classes: 10
- n_epochs: 500
- lr: 0.1
- optimizer: sgd
- batch_size: 64
- num_workers: 8
- img_size: 32
- strategy: CoNAL[scale=0]
- scheduler: True
- milestones: (250, 400)
- dataset: /home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H
- pretrained: False
- momentum: 0.9
- decay: 0.0005
- n_params: 3072
- lr_decay: 0.1
- data_augmentation: False
- path_remove: None
- metadata_path: None
- freeze: False
----------
Loading datasets
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision

# WAUM identification

In [61]:
!peerannot identify $DIRc10H -K 10 --method WAUM \
                    --labels $DIR_answers --model resnet18 \
                    --n-epochs 200 --lr=0.1 --img-size=4 --maxiter-DS=100 --alpha=0.01

Running the following configuration:
----------
- Data at /home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H
- number of classes: 10
- labels: /home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H/answers.json
- model: resnet18
- n_epochs: 2
- lr: 0.1
- img_size: 4
- maxiter_ds: 10
- alpha: 0.01
- use_pleiss: False
- n_params: 3072
- pretrained: False
- momentum: 0.9
- metadata_path: None
- decay: 0.0005
- optimizer: SGD
- data_augmentation: False
----------
Train set: 488448 tasks
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Successfully loaded resnet18 with n_classes=10
Removing initial downsampling
Running identification with method: WAUMstacked
computing WAUM:   0%|                                  | 0/9500 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/home/tlefort/condaenvs/phd/b

# AUM identification

In [59]:
!peerannot identify $DIRc10H -K 10 --method AUM \
                    --model resnet18 \
                    --n-epochs 200 --lr=0.1 --img-size=32

Running the following configuration:
----------
- Data at /home/tlefort/Documents/peerannot/peerannot/datasets/cifar10H
- number of classes: 10
- model: resnet18
- n_epochs: 2
- lr: 0.1
- img_size: 32
- labels: /home/tlefort/Documents/peerannot/peerannot/examples/answers.json
- use_pleiss: False
- alpha: 0.01
- n_params: 3072
- pretrained: False
- momentum: 0.9
- metadata_path: None
- decay: 0.0005
- maxiter_ds: 50
- optimizer: SGD
- data_augmentation: False
----------
Train set: 9500 tasks
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Successfully loaded resnet18 with n_classes=10
Removing initial downsampling
Running identification with method: AUM
['bird-1038.png' 'bird-1039.png' 'bird-1043.png' ... 'truck-970.png'
 'truck-979.png' 'truck-981.png'] (9500,)
Saved full log at /home/tlefort/Documents/peerannot/peeranno